In [1]:
import pandas as pd
import numpy as np
import functions as ff
import math
import random
from collections import Counter
import csv   
import os
from bayes_opt import BayesianOptimization

In [2]:
pair = "gbpjpy"
start_date = 2018
end_date = 2018
freq = 30

file_csv = '{}_{}_{}_{}.csv'.format(pair.upper(), freq, start_date, end_date)
file_csv_temp = '{}_{}_{}_{}Temp.csv'.format(pair.upper(), freq, start_date, end_date)
file_csv_clean = '{}_{}_{}_{}Clean.csv'.format(pair.upper(), freq, start_date, end_date)

curr = os.getcwd()
if not os.path.split(curr)[1]=='FX_data': os.chdir("./FX_data/")
if os.path.exists(file_csv_clean):
    data = pd.read_csv(file_csv_clean)
else:
    data = ff.construct_data(pair,start_date,end_date,freq)
    data.to_csv(file_csv_clean)

Close = data["Close"]
High = data["High"]
Low = data["Low"]
Time = data["Time"]
Volume = data["Volume"]

stochastic = ff.sstoc(data)
N = data.shape[0]

performance = []
limit_levels = []
stop_levels = []
distance_levels = []
min_levels = []
trailing_stop_levels = []

#PIP_RATIO = math.pow(10,-len(str(Close[0]).split('.')[1])+1)

sample = random.sample(list(Close.values), 1000)

pip_list = []
for i in range(1000):
    pip_list.append(math.pow(10,-len(str(sample[i]).split('.')[1])+1))

c = Counter(pip_list)
PIP_RATIO, count = c.most_common()[0]


INVERSE_PIP_RATIO = 1/PIP_RATIO
MARGIN = 4

def distant(first,second, Distance = 50):
    if((first-second)> Distance * PIP_RATIO):
        return True
    else:
        return False

In [7]:
def performace(limit, stop, distance,trailing_stop):
    first_min = 1000
    first_max = 0
    second_min = 1000
    second_max = 0
    p = ff.position(PIP_RATIO)
    
    cumpips = 0
    state = -3
    uptrend = False
    max_value = 0
    min_value = 1000

    for i in range(N):
        if p.have_position:
                cumpips, state = p.close_position(cumpips, Time[i], Low[i], High[i], state)
                    
        if state == -3:
                if first_max < High[i]: first_max = High[i]
                if first_min > Low[i]: first_min = Low[i]
                    
                SC = ff.stochastic_crossover(i, stochastic)
                if SC==2: 
                    state = -21
                    first_min = first_max
                elif SC==1:
                    state = -20
                    first_max = first_min
        
        elif state == -20:
                if first_max < High[i]: first_max = High[i]
                    
                SC = ff.stochastic_crossover(i, stochastic)
                if SC==2: 
                    state = -22
                    first_min = first_max
                    
        elif state == -22:
                if first_min > Low[i]: first_min = Low[i]
                    
                SC = ff.stochastic_crossover(i, stochastic)
                if SC==1: 
                    state = -24
                    second_max = first_min
                    
        elif state == -24:
                if second_max < High[i]: second_max = High[i]
                    
                SC = ff.stochastic_crossover(i, stochastic)
                if SC==2: 
                    if second_max <= first_max:
                        uptrend = False
                        triggerprice = first_max
                        trendprice = first_min
                        state = 0
                    else:
                        uptrend = True
                        triggerprice = first_min
                        trendprice = second_max
                        state = 0
    
        elif state == -21:
                if first_min > Low[i]: first_min = Low[i]
                    
                SC = ff.stochastic_crossover(i, stochastic)
                if SC==1: 
                    state = -23
                    first_max = first_min
                    
        elif state == -23:
                if first_max < High[i]: first_max = High[i]
                    
                SC = ff.stochastic_crossover(i, stochastic)
                if SC==2: 
                    state = -25
                    second_min = first_max
                    
        elif state == -25:
                if second_min > Low[i]: second_min = Low[i]
                    
                SC = ff.stochastic_crossover(i, stochastic)
                if SC==2: 
                    if second_min <= first_min:
                        uptrend = False
                        triggerprice = first_max
                        trendprice = second_min
                        state = 0
                    else:
                        uptrend = True
                        triggerprice = first_min
                        trendprice = first_max
                        state = 0        
    
        elif state == 0:
            if uptrend:
                
                if max_value < High[i]: max_value = High[i]
                if min_value > Low[i]: min_value = Low[i]
                
                if distant(max_value,trendprice, distance):
                    triggerprice = min_value
                    #min_value = High[i]
                    state = 10
                  
                if distant(triggerprice,min_value, distance):  
                    state = 1
                    max_value = min_value
            else: 
                
                if max_value < High[i]: max_value = High[i]
                if min_value > Low[i]: min_value = Low[i]
                
                if distant(trendprice, min_value, distance):
                    triggerprice = max_value
                    #max_value = Low[i]
                    state = 10
                
                if distant(max_value,triggerprice, distance):
                    state = 1
                    min_value = max_value
        
        elif state == 10:  
            if uptrend:
                
                if max_value < High[i]: max_value = High[i]
                if min_value > Low[i]: min_value = Low[i]
    
                SC = ff.stochastic_crossover(i, stochastic)
                if SC==2:
                    trendprice = max_value 
                   # triggerprice = min_value
                    min_value = High[i]
                    state = 0
                
                if distant(triggerprice,min_value, distance):
                    trendprice = max_value
                    state = 1
            else:
                
                if max_value < High[i]: max_value = High[i]
                if min_value > Low[i]: min_value = Low[i]
                
                SC = ff.stochastic_crossover(i, stochastic)
                if SC == 1:
                    trendprice = min_value
                  #  triggerprice = max_value
                    max_value = Low[i]
                    state = 0
                    
                if distant(max_value,triggerprice, distance):
                    trendprice = min_value
                    state = 1
    
        elif state == 1:
            if uptrend:
                
                if max_value < High[i]: max_value = High[i]
                if min_value > Low[i]: min_value = Low[i]
                
                if distant(max_value,trendprice, distance):
                    if p.have_position:
                        state = 3
                        continue                
                    else:
                        state = 0
                        continue
                
                SC = ff.stochastic_crossover(i, stochastic)
                if SC==1: 
                    shootingprice = min_value
                    max_value = min_value
                    state = 2
            else:
                
                if max_value < High[i]: max_value=High[i]
                if min_value > Low[i]: min_value=Low[i]
                
                if distant(trendprice,min_value, distance):
                    if p.have_position:
                        state = 3
                        continue 
                    else:
                        state = 0
                        continue
                
                SC = ff.stochastic_crossover(i, stochastic)
                if SC==2:
                    shootingprice = max_value
                    min_value = max_value
                    state = 2 
        
        elif state == 2:
            
            if uptrend:
                
                if max_value < High[i]: max_value=High[i]
                if min_value > Low[i]: min_value=Low[i]
                
                if distant(max_value,trendprice, distance):
                    if p.have_position:
                        triggerprice = min_value
                        state = 13
                        min_value = High[i]
                        continue
                    else:
                        triggerprice = shootingprice
                        state = 10
                        min_value = High[i]
                        continue
                
                if distant(shootingprice,min_value, distance):
                    limit_price = shootingprice - (distance + limit) * PIP_RATIO
                    stop_price = shootingprice  - (distance - stop) * PIP_RATIO
                    uptrend, cumpips = p.take_position("Sell",uptrend,cumpips,shootingprice - distance * PIP_RATIO, abs(shootingprice - trendprice)*100, abs(triggerprice - trendprice)*100, Time[i], limit_price, stop_price, trailing_stop)
                    state = 3
                    trendprice = shootingprice
                    triggerprice = max_value
                    continue
            else:
                if max_value<High[i]: max_value=High[i]
                if min_value>Low[i]: min_value=Low[i];
                
                if distant(trendprice, min_value, distance):
                    if p.have_position:
                        triggerprice = max_value
                        state = 13
                        max_value = Low[i]
                        continue                 
                    else:
                        triggerprice = shootingprice
                        state = 10
                        max_value = Low[i]
                        continue
                
                if distant(max_value,shootingprice, distance):
                    limit_price = shootingprice + (distance + MARGIN + limit) * PIP_RATIO
                    stop_price = shootingprice + (distance  - stop) * PIP_RATIO
                    uptrend, cumpips = p.take_position("Buy",uptrend,cumpips,shootingprice + (distance + MARGIN) * PIP_RATIO, abs(shootingprice - trendprice)*100, abs(triggerprice - trendprice)*100, Time[i], limit_price, stop_price, trailing_stop)
                    state = 3
                    trendprice = shootingprice
                    triggerprice = min_value
                    continue
        
        elif state == 3:
            if uptrend:
                if max_value < High[i]: max_value = High[i]
                if min_value > Low[i]: min_value = Low[i]
                        
                if distant(max_value,trendprice, distance):
                    triggerprice = min_value
                    state = 13
                  
                if distant(triggerprice,min_value, distance):  
                    state = 1 
            else: 
                
                if max_value < High[i]: max_value = High[i]
                if min_value > Low[i]: min_value = Low[i]
                
                if distant(trendprice, min_value, distance):
                    triggerprice = max_value
                    state = 13
                
                if distant(max_value,triggerprice, distance):
                    state = 1
                    continue
        
        elif state == 13:  
            if uptrend:
                
                if max_value < High[i]: max_value = High[i]
                if min_value > Low[i]: min_value = Low[i]
    
                SC = ff.stochastic_crossover(i, stochastic)
                if SC==2:
                    trendprice = max_value
                    min_value = High[i]
                    state = 3
                
                if distant(triggerprice,min_value, distance):
                    trendprice = max_value
                    state = 1
                    continue
            
            else:
                
                if max_value < High[i]: max_value = High[i]
                if min_value > Low[i]: min_value = Low[i]
                
                SC = ff.stochastic_crossover(i, stochastic)
                if SC == 1:
                    trendprice = min_value
                    max_value = Low[i]
                    state = 3
                    
                if distant(max_value,triggerprice, distance):
                    trendprice = min_value
                    state = 1 
                    continue
    
    return(cumpips)
    del p
 

In [8]:
# Bounded region of parameter space
pbounds = {'limit': (100, 3100), 'stop': (100, 3100),'distance': (10, 1000),'trailing_stop': (0, 500)}

optimizer = BayesianOptimization(
    f=performace,
    pbounds=pbounds,
    random_state=1,
)

In [10]:
optimizer.maximize(
    init_points=2,
    n_iter=100,
)

|   iter    |  target   | distance  |   limit   |   stop    | traili... |
-------------------------------------------------------------------------
|  6        |  0.0      |  402.8    |  1.716e+0 |  1.358e+0 |  342.6    |
|  7        |  0.0      |  212.4    |  2.734e+0 |  182.2    |  335.2    |
|  8        |  0.0      |  212.3    |  2.734e+0 |  182.3    |  335.9    |
|  9        |  0.0      |  404.3    |  2.302e+0 |  108.0    |  168.5    |
|  10       |  0.0      |  225.3    |  2.701e+0 |  178.7    |  334.9    |
|  11       |  0.0      |  413.6    |  2.284e+0 |  101.1    |  154.7    |
|  12       |  0.0      |  414.3    |  2.27e+03 |  117.8    |  181.7    |
|  13       |  0.0      |  415.0    |  2.276e+0 |  101.8    |  172.6    |
|  14       |  0.0      |  405.8    |  2.274e+0 |  121.0    |  159.3    |
|  15       |  0.0      |  436.5    |  2.287e+0 |  123.6    |  163.7    |
|  16       |  0.0      |  219.2    |  2.717e+0 |  190.6    |  332.7    |
|  17       |  0.0      |  997.7    | 

|  72       |  0.0      |  453.8    |  2.313e+0 |  138.3    |  144.5    |
|  73       |  0.0      |  461.1    |  2.319e+0 |  141.0    |  185.0    |
|  74       |  0.0      |  421.0    |  1.699e+0 |  1.361e+0 |  332.7    |
|  75       | -361.0    |  196.0    |  2.683e+0 |  178.5    |  344.4    |
|  76       |  0.0      |  387.2    |  1.733e+0 |  1.305e+0 |  330.6    |
|  77       |  0.0      |  471.5    |  2.327e+0 |  141.8    |  204.6    |
|  78       |  0.0      |  211.8    |  2.737e+0 |  177.0    |  328.0    |
|  79       |  0.0      |  463.2    |  2.234e+0 |  137.8    |  172.7    |
|  80       |  0.0      |  405.7    |  2.252e+0 |  108.3    |  141.3    |
|  81       |  0.0      |  428.3    |  2.255e+0 |  108.1    |  137.0    |
|  82       |  0.0      |  443.9    |  2.271e+0 |  124.9    |  188.5    |
|  83       |  0.0      |  444.8    |  2.25e+03 |  129.5    |  168.8    |
|  84       |  0.0      |  385.4    |  1.73e+03 |  1.37e+03 |  338.4    |
|  85       |  0.0      |  431.4    | 

In [11]:
print(optimizer.max)

{'target': 0.0, 'params': {'distance': 422.8517846555483, 'limit': 2260.973480326474, 'stop': 100.34312445203466, 'trailing_stop': 151.16628631591988}}
